In [1]:
from morty.pitchdistribution import PitchDistribution as pd
from morty.chordia import Chordia
from morty.evaluator import Evaluator
import os
import json


In [2]:
# Initializations
ev = Evaluator()
pitch_path = "data"
mode_list = ["Hicaz", "Huseyni", "Saba"]
train_dir = "train_dir"

# Metadata of Leave-One-Out Cross Validation Folds
annotations = json.load(open(os.path.join(pitch_path, 'annotations.json'), 'r'))
tonic_annotations = {annot['mbid']: annot['tonic'] for annot in annotations}
folds = [json.load(open(os.path.join("folds", ("fold_%d.json"%i)), 'r')) 
         for i in range(1,6)]
test_folds = [fold['test'] for fold in folds]
train_folds = [fold['train'] for fold in folds]

# Parameter Set of the Experiment
step_size = 7.5
smooth_factor = 15
distribution_type = "pcd"
distance_method = "bhat"
k_param = 3
chordia = Chordia(step_size=step_size, smooth_factor=smooth_factor, chunk_size=0)

# Identifier String of this Parameter Set
exp_str = ("sf_%2.1f__ss_%2.1f__%s__dist_%s__k_%d" % 
           (smooth_factor, step_size, distribution_type, 
            distance_method, k_param)).replace('.', '')


In [3]:
for fold_idx in range(5):
    train_fold = train_folds[fold_idx]
    test_fold = test_folds[fold_idx]
    
    # Training
    for mode in mode_list:            
        train_mbids = [track['mbid'] for track in train_fold if(track['makam'] == mode)]
        train_tonics = [tonic_annotations[mbid] for mbid in train_mbids]
        train_paths = [os.path.join(pitch_path, mode, (mbid+".pitch")) 
                       for mbid in train_mbids]
        
        chordia.train(mode, train_paths, train_tonics, metric=distribution_type, 
                      save_dir=train_dir)
    
    # Testing    
    for test_pitch in test_fold:
        test_path = os.path.join(pitch_path, test_pitch["makam"], 
                                 (test_pitch["mbid"]+".pitch"))
        
        # Mode Recognition
        mode_est = chordia.mode_estimate(
            test_path, test_pitch["tonic"], mode_names=mode_list,
            mode_dir=train_dir, distance_method=distance_method,
            metric=distribution_type, k_param=k_param)
        
        # Tonic Identification
        tonic_est = chordia.tonic_estimate(
            test_path, mode, mode_dir=train_dir, 
            distance_method=distance_method, metric=distribution_type, 
            k_param=k_param)
        
        # Joint Estimation
        joint_tonic_est, joint_mode_est = chordia.joint_estimate(
            test_path, mode_list, mode_dir=train_dir, 
            distance_method=distance_method, metric=distribution_type, 
            k_param=k_param)
        
        # Evaluations
        mode_eval = (mode_est == test_pitch["makam"])
        tonic_eval = ev.tonic_evaluate(test_pitch["mbid"], tonic_est, 
                                       test_pitch["tonic"])["tonic_eval"]
        joint_eval = ((mode_est == test_pitch["makam"]), 
                      ev.tonic_evaluate(test_pitch["mbid"], joint_tonic_est,
                                        test_pitch["tonic"])["tonic_eval"])
        
        print("mbid:%s" % test_pitch["mbid"])
        print("\nMode Recognition:\tEstimated:%s\tActual:%s\tEvaluation:%s" % 
              (mode_est, test_pitch["makam"], str(mode_eval)))
        print("Tonic Identification:\tEstimated:%4.2f\tActual:%4.2f\tEvaluation:%s\n" % 
              (tonic_est, test_pitch["tonic"], str(tonic_eval)))
        print(("Joint Estimation:\nMode:\tEstimated:%s\tActual:%s\tEvaluation:%s\n"
               "Tonic:\tEstimated:%4.2f\tActual:%4.2f\tEvaluation:%s\n") % 
              (joint_mode_est, test_pitch["makam"], str(joint_eval[0]),
               joint_tonic_est, test_pitch["tonic"], str(joint_eval[1])))
        print("----------------------------------------------------------------------")


mbid:2c88acdf-685d-42c7-913d-1a9f2005587e

Mode Recognition:	Estimated:Hicaz	Actual:Hicaz	Evaluation:True
Tonic Identification:	Estimated:622.25	Actual:292.50	Evaluation:False

Joint Estimation:
Mode:	Estimated:Hicaz	Actual:Hicaz	Evaluation:True
Tonic:	Estimated:585.64	Actual:292.50	Evaluation:True

----------------------------------------------------------------------
mbid:0f0e4bc3-67f9-4727-818b-983320e897cb

Mode Recognition:	Estimated:Saba	Actual:Saba	Evaluation:True
Tonic Identification:	Estimated:903.17	Actual:225.60	Evaluation:True

Joint Estimation:
Mode:	Estimated:Saba	Actual:Saba	Evaluation:True
Tonic:	Estimated:903.17	Actual:225.60	Evaluation:True

----------------------------------------------------------------------
mbid:0eac190d-13c4-442f-bb13-cf734d3cbe88

Mode Recognition:	Estimated:Hicaz	Actual:Huseyni	Evaluation:False
Tonic Identification:	Estimated:883.82	Actual:221.00	Evaluation:True

Joint Estimation:
Mode:	Estimated:Hicaz	Actual:Huseyni	Evaluation:False
Tonic:	Est